In [1]:
import sys
import os
sys.path.append(os.getcwd()[:-8])

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import plotly.io as pio
pio.renderers.default = 'colab'

In [4]:
from ACME.ACME import ACME
import numpy as np
import pandas as pd

In [5]:
mean = [5,5,5,5,5,5]
cov = np.identity(6)
x_normal = pd.DataFrame(np.random.multivariate_normal(mean, cov, 5000), columns = ['f'+str(i) for i in range(1,7)] )
x_normal['is_out'] = 0

In [6]:
mean = [10,10,8,5,5,5]
cov = np.identity(6)
x_outliers =  pd.DataFrame(np.random.multivariate_normal(mean, cov, 200), columns = ['f'+str(i) for i in range(1,7)] )
x_outliers['is_out'] = 1

In [7]:
X = pd.concat([x_normal,x_outliers],ignore_index=True, sort=False).reset_index(drop=True)

In [8]:
features = ['f'+str(i) for i in range(1,7)]

In [9]:
from sklearn.ensemble import IsolationForest
if_model = IsolationForest(random_state=0).fit(X[features])

In [10]:
X['pred_AD_score'] = if_model.decision_function(X[features])
X['pred_AD_score'] = -1*X['pred_AD_score']
X.tail()

,f1,f2,f3,f4,f5,f6,is_out,pred_AD_score
5195,9.840582,9.540175,6.701149,5.794398,6.309258,4.603287,1,0.102451
5196,10.917180,8.512942,8.753023,5.328947,4.103141,5.469347,1,0.108566
5197,10.265241,12.797233,6.320346,4.456850,4.328865,6.633918,1,0.159123
5198,9.980945,10.913432,8.603964,6.523390,3.133931,5.473578,1,0.155328
5199,10.038284,10.135751,7.447155,5.692496,3.415625,4.317029,1,0.130621


In [11]:
X['pred_AD_score'].plot(kind='hist')

<AxesSubplot: ylabel='Frequency'>

In [12]:
bottom_anomalies = X.loc[(X['pred_AD_score']>0.01)].sort_values(by='pred_AD_score', ascending=False).tail(10)
top_anomalies = X.loc[(X['pred_AD_score']>0.01)].sort_values(by='pred_AD_score', ascending=False).head(10)

In [13]:
top_normal = X.loc[(X['pred_AD_score']<0.01)].sort_values(by='pred_AD_score', ascending=False).tail(10)
bottom_normal = X.loc[(X['pred_AD_score']<0.01)].sort_values(by='pred_AD_score', ascending=False).head(10)

In [14]:
top_anomalies

,f1,f2,f3,f4,f5,f6,is_out,pred_AD_score
5144,11.058264,11.927793,8.779367,3.609303,4.649548,6.501098,1,0.184216
5006,10.155152,10.227756,9.392364,2.117623,5.668017,4.842743,1,0.180655
5051,11.061485,11.969076,7.367044,4.145871,4.333509,7.432336,1,0.180102
5015,11.312698,10.502687,9.951109,5.405401,7.388789,5.441224,1,0.178799
5003,10.643397,9.811537,9.800488,3.438618,6.069876,3.473033,1,0.175082
5142,9.365312,12.496451,8.592090,3.720781,5.306769,6.407538,1,0.169979
5102,10.237254,11.224582,9.089462,5.657276,7.151683,4.419258,1,0.169235
5122,12.012866,10.320615,9.711006,3.576390,5.251762,3.731179,1,0.167741
5151,9.762638,9.530349,10.282820,3.842351,3.525996,3.317227,1,0.166830
5108,11.592897,9.700228,8.870110,6.113755,3.798045,6.201587,1,0.166198


In [15]:
k = 20
def score_function(model, data):
    try: # for global
        df = model.decision_function(data)
    except: # for local
        df = model.decision_function(data.reshape(1,-1))
    
    return -1*df

acme_ifo = ACME(model = if_model, target = 'AD_score', K=k, task='ad', score_function=score_function, features=features)
acme_ifo = acme_ifo.explain(X[features], robust = True)

In [16]:
acme_local = acme_ifo.explain_local(X[features].loc[bottom_anomalies.index[-1]])

In [17]:
acme_local.summary_table(local=True)

,quantile,original,type_feature,predict,baseline_prediction,effect,baseline_quantile,class,importance
feature,,,,,,,,,
f1,0.730962,5.682703,numeric,0.010465,0.010465,0.000000,0.730962,None,0.006385
f1,0.050000,3.348065,numeric,0.025420,0.010465,0.014954,0.730962,None,0.006385
f1,0.100000,3.712842,numeric,0.018681,0.010465,0.008216,0.730962,None,0.006385
f1,0.150000,3.968783,numeric,0.014177,0.010465,0.003712,0.730962,None,0.006385
f1,0.200000,4.185394,numeric,0.014453,0.010465,0.003988,0.730962,None,0.006385
...,...,...,...,...,...,...,...,...,...
f6,0.750000,5.648394,numeric,0.010687,0.010465,0.000221,0.179808,None,0.007008
f6,0.800000,5.813393,numeric,0.017101,0.010465,0.006636,0.179808,None,0.007008
f6,0.850000,5.998447,numeric,0.024456,0.010465,0.013991,0.179808,None,0.007008


In [18]:
acme_local.feature_importance()

,importance
feature,
f2,0.028938
f4,0.028325
f3,0.027615
f1,0.023608
f6,0.022618
f5,0.017979


In [19]:
acme_local.feature_importance(local=True, weights = {'ratio':0.1,'delta':0.1,'change':0.3,'distance':0.5})

,ratio,delta,change,distance_to_change,max_score,min_score,local_score,importance
f4,1.000000,0.079733,1.0,0.048462,0.010465,-0.069267,0.010465,0.883742
f5,1.000000,0.034194,1.0,0.069808,0.010465,-0.023728,0.010465,0.868516
f3,0.467881,0.043204,1.0,0.147308,0.033455,-0.009749,0.010465,0.777455
f6,0.221247,0.038822,0.0,1.000000,0.040698,0.001876,0.010465,0.026007
f1,0.100122,0.049822,0.0,1.000000,0.055299,0.005477,0.010465,0.014994
f2,0.039630,0.045776,0.0,1.000000,0.054427,0.008651,0.010465,0.008541


In [20]:
acme_local.feature_importance(local=True, weights = {'ratio':0.4,'delta':0.1,'change':0.3,'distance':0.2})

,ratio,delta,change,distance_to_change,max_score,min_score,local_score,importance
f4,1.000000,0.079733,1.0,0.048462,0.010465,-0.069267,0.010465,0.898281
f5,1.000000,0.034194,1.0,0.069808,0.010465,-0.023728,0.010465,0.889458
f3,0.467881,0.043204,1.0,0.147308,0.033455,-0.009749,0.010465,0.662011
f6,0.221247,0.038822,0.0,1.000000,0.040698,0.001876,0.010465,0.092381
f1,0.100122,0.049822,0.0,1.000000,0.055299,0.005477,0.010465,0.045031
f2,0.039630,0.045776,0.0,1.000000,0.054427,0.008651,0.010465,0.020429


In [21]:
acme_local.feature_exploration('f1',local=True)

,quantile,original,type_feature,predict,baseline_prediction,effect,baseline_quantile,class,size,importance,direction
feature,,,,,,,,,,,
f1,0.730962,5.682703,numeric,0.010465,0.010465,0.000000,0.730962,None,0.30,0.006385,anomalies
f1,0.050000,3.348065,numeric,0.025420,0.010465,0.014954,0.730962,None,0.05,0.006385,anomalies
f1,0.100000,3.712842,numeric,0.018681,0.010465,0.008216,0.730962,None,0.05,0.006385,anomalies
f1,0.150000,3.968783,numeric,0.014177,0.010465,0.003712,0.730962,None,0.05,0.006385,anomalies
f1,0.200000,4.185394,numeric,0.014453,0.010465,0.003988,0.730962,None,0.05,0.006385,anomalies
f1,0.250000,4.351127,numeric,0.013757,0.010465,0.003292,0.730962,None,0.05,0.006385,anomalies
f1,0.300000,4.516638,numeric,0.012211,0.010465,0.001746,0.730962,None,0.05,0.006385,anomalies
f1,0.350000,4.650179,numeric,0.011422,0.010465,0.000957,0.730962,None,0.05,0.006385,anomalies
f1,0.400000,4.783258,numeric,0.009968,0.010465,-0.000497,0.730962,None,0.05,0.006385,anomalies


In [22]:
acme_local.feature_exploration('f2',local=True,plot=True)

In [23]:
acme_local.baseline_values()

,f1,f2,f3,f4,f5,f6
0,5.184302,5.223028,5.082801,4.997299,5.013666,4.984476


In [24]:
acme_local.summary_plot(local=True)

In [25]:
acme_local.summary_table(local=True)

,quantile,original,type_feature,predict,baseline_prediction,effect,baseline_quantile,class,importance
feature,,,,,,,,,
f1,0.730962,5.682703,numeric,0.010465,0.010465,0.000000,0.730962,None,0.006385
f1,0.050000,3.348065,numeric,0.025420,0.010465,0.014954,0.730962,None,0.006385
f1,0.100000,3.712842,numeric,0.018681,0.010465,0.008216,0.730962,None,0.006385
f1,0.150000,3.968783,numeric,0.014177,0.010465,0.003712,0.730962,None,0.006385
f1,0.200000,4.185394,numeric,0.014453,0.010465,0.003988,0.730962,None,0.006385
...,...,...,...,...,...,...,...,...,...
f6,0.750000,5.648394,numeric,0.010687,0.010465,0.000221,0.179808,None,0.007008
f6,0.800000,5.813393,numeric,0.017101,0.010465,0.006636,0.179808,None,0.007008
f6,0.850000,5.998447,numeric,0.024456,0.010465,0.013991,0.179808,None,0.007008
